In [ ]:
#import denpendencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

from config import census_key


In [ ]:
#read csv files
populationcsv = "Data/Top100_CitiesPop.csv"
population = pd.read_csv(populationcsv)


In [ ]:
#read density csv file from simplemaps.com
densitycsv = "Data/top_100.csv"
density = pd.read_csv(densitycsv)

In [ ]:
#merge the files on City
popdens = pd.merge(population, density, how="left", left_on="City", right_on="city")
popdens_clean = popdens[["Rank", "City", "2010 Census", "state", "pop_density"]]
popdens_clean = popdens_clean.rename(columns = {"pop_density": "Persons by km2", "state": "State"})

In [ ]:
# Beginning of Housing Type/Cost Data Cleaning

In [ ]:
# Import data table -need leading zeroes for FIPS
api_cities = pd.read_csv("Data/Top100_FIPSCodes.csv", dtype={'StateFIPS': 'str', 'Place FIPS': 'str'})

In [ ]:
# Adding columns to data frame for API call
api_cities["Name"] = ''
api_cities["B25024_001E"] = ""
api_cities["B25024_002E"] = ""
api_cities["B25024_003E"] = ""
api_cities["B25024_004E"] = ""
api_cities["B25024_005E"] = ""
api_cities["B25024_006E"] = ""
api_cities["B25024_007E"] = ""
api_cities["B25024_008E"] = ""
api_cities["B25024_009E"] = ""
api_cities["B25024_010E"] = ""
api_cities["B25024_011E"] = ""
api_cities["B25105_001E"] = ""
api_cities["B25105_001M"] = ""

In [ ]:
# API Call
# Housing (Units and structure) - B25024_001E through B25024_011E
# Meidan Housing Costs - B25105_001E (Estimate) and B25105_001M (Margin of error)
variables = "NAME,B25024_001E,B25024_002E,B25024_003E,B25024_004E,B25024_005E,B25024_006E,B25024_007E,\
B25024_008E,B25024_009E,B25024_010E,B25024_011E,B25105_001E,B25105_001M"

# base URL
base_url = "https://api.census.gov/data/2018/acs/acs1?get="

# API Call - looping through data frame
for index, row in cities.iterrows():
    
    st = row["StateFIPS"]
    place = row["Place FIPS"]
    
    query_url = f"{base_url}{variables}&for=place:{place}&in=state:{st}&key={census_key}"
    
    try:
        response = requests.get(query_url)
        housing_call = response.json()
    
        # Update dataframe
        api_cities.loc[index, "Name"] = housing_call[1][0]
        api_cities.loc[index, "B25024_001E"] = housing_call[1][1]
        api_cities.loc[index, "B25024_002E"] = housing_call[1][2]
        api_cities.loc[index, "B25024_003E"] = housing_call[1][3]
        api_cities.loc[index, "B25024_004E"] = housing_call[1][4]
        api_cities.loc[index, "B25024_005E"] = housing_call[1][5]
        api_cities.loc[index, "B25024_006E"] = housing_call[1][6]
        api_cities.loc[index, "B25024_007E"] = housing_call[1][7]
        api_cities.loc[index, "B25024_008E"] = housing_call[1][8]
        api_cities.loc[index, "B25024_009E"] = housing_call[1][9]
        api_cities.loc[index, "B25024_010E"] = housing_call[1][10]
        api_cities.loc[index, "B25024_011E"] = housing_call[1][11]
        api_cities.loc[index, "B25105_001E"] = housing_call[1][12]
        api_cities.loc[index, "B25105_001M"] = housing_call[1][13]
    
    except:
        print("City notfound")

api_cities

In [ ]:
# Rename Columns
housing = api_cities.rename(columns={"B25024_001E": "Total Response",
                              "B25024_002E": "1, detached",
                              "B25024_003E": "1, attached",
                              "B25024_004E": "2 units",
                              "B25024_005E": "3 or 4 units",
                              "B25024_006E": "5 to 9 units",
                              "B25024_007E": "10 to 19 units",
                              "B25024_008E": "20 to 49 units",
                              "B25024_009E": "50 or more units",
                              "B25024_010E": "Mobile Home",
                              "B25024_011E": "Boat RV Van Etc",
                              "B25105_001E": "Median Housing Costs",
                              "B25105_001M": "MOE Housing Costs"})

In [ ]:
# New Column for UGB identifier
housing["UGB"] = ""

# Create list for UGB identifer
ugb = pd.read_csv("Data/ugb_cities.csv")
ugb_cities = list(ugb['City'])

# Add UGB Identifier to Dataframe
for index, row in housing.iterrows():
    if row["City"] in ugb_cities:
        housing.loc[index, "UGB"] = "Yes"
    else: housing.loc[index, "UGB"] = "No"


In [ ]:
# Changing API call data type from object to a number
housing.dtypes
housing = housing.apply(pd.to_numeric, errors='ignore')

In [ ]:
# Adding the 1 unit columns together
housing["1 unit"] = housing["1, detached"] + housing["1, attached"]

# Creating other group for non structured types
housing["Other"] = housing["Mobile Home"] + housing["Boat RV Van Etc"]

# Filtering out non-necessary columns and rearranging
housing_clean = housing[["City", "State", "UGB", "1 unit", "2 units", "3 or 4 units", "5 to 9 units",
                    "10 to 19 units", "20 to 49 units", "50 or more units", "Other", "Total Response", 
                         "Median Housing Costs", "MOE Housing Costs"]]

# Output to csv
housing_clean.to_csv("Data/HousingTypes_Cleaned.csv", index=False, header=True)

In [ ]:
# End of Housing Type/Cost Data Cleaning